# 协同过滤：求用户的相似度，基于用户的方法

In [14]:
#需要安装surprise库：pip install surprise,但是在此之前需要先安装Microsoft C++
from surprise import KNNBasic,SVD
# KNNBasic: 是相当于是协同过滤算法
# SVD是基于矩阵分解的，对应于隐语义模型
from surprise import Dataset
from surprise import model_selection, print_perf
# 官网：http://surprise.readthedocs.io/en/stable/index.html
# 测试的数据集下载： http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

# Load the movielens-100k dataset (download it if needed),
# and split it into 3 folds for cross-validation.
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

# We'll use the famous KNNBasic algorithm.
algo = KNNBasic() #实例化算法对象

# Evaluate performances of our algorithm on the dataset.
perf = model_selection.cross_validate(algo, data, measures=['RMSE', 'MAE'])
#算法名字，数据，评估标准，RMSE：均方误差，MAE绝对误差

print_perf(perf)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
        Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    
TEST_RMSE0.9763  0.9810  0.9847  0.9697  0.9811  0.9786  
TEST_MAE0.7719  0.7764  0.7758  0.7671  0.7734  0.7729  
FIT_TIME0.5285  0.5552  0.5495  0.5521  0.5555  0.5482  
TEST_TIME3.3078  3.3966  3.4591  3.5935  3.4465  3.4407  


# 隐语义模型

In [44]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

# Use movielens-100K
data = Dataset.load_builtin('ml-100k')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]} #正则化的系数
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'fcp'], cv=3)

gs.fit(data)

0.9640828425436142
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [49]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# best FCP score
print(gs.best_score['fcp'])

# combination of parameters that gave the best FCP score
print(gs.best_params['fcp'])


0.9640828425436142
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
0.6978407920875513
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [46]:
import pandas as pd  

results_df = pd.DataFrame.from_dict(grid_search.cv_results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_fcp,split1_test_fcp,...,std_test_fcp,rank_test_fcp,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,0.992955,0.993510,0.990746,0.984586,0.992969,0.990953,0.003322,7,0.678077,0.684572,...,0.002528,1,1.134566,0.002721,0.157379,0.035611,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,0.999282,1.000517,0.997457,0.991284,1.000033,0.997715,0.003379,8,0.680027,0.686586,...,0.003045,2,1.115019,0.004505,0.154786,0.035263,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6
2,0.971983,0.970754,0.969922,0.963016,0.972885,0.969712,0.003498,2,0.686659,0.692536,...,0.002110,5,1.116019,0.003280,0.152788,0.034042,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}",5,0.005,0.4
3,0.981136,0.980838,0.978914,0.972058,0.982050,0.978999,0.003618,5,0.686049,0.693653,...,0.002822,6,1.112027,0.002601,0.153390,0.034792,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}",5,0.005,0.6
4,0.975948,0.974880,0.973863,0.967261,0.976417,0.973674,0.003326,4,0.684040,0.690164,...,0.002899,3,2.218069,0.010498,0.168350,0.033647,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}",10,0.002,0.4
5,0.984172,0.984343,0.982248,0.975544,0.985094,0.982280,0.003497,6,0.684864,0.693260,...,0.003057,4,2.211487,0.002148,0.152593,0.035018,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}",10,0.002,0.6
6,0.963872,0.961850,0.961374,0.954271,0.964239,0.961121,0.003600,1,0.689618,0.697427,...,0.002703,7,2.224848,0.009384,0.153390,0.034430,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}",10,0.005,0.4
7,0.973891,0.972779,0.971317,0.964318,0.974422,0.971345,0.003671,3,0.688998,0.698096,...,0.002942,8,2.216274,0.004155,0.167753,0.035598,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}",10,0.005,0.6


# 推荐电影

In [55]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import os
import io

from surprise import KNNBaseline
from surprise import Dataset


def read_item_names():


    file_name = ('./ml-100k/u.item')
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid



data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset() #将数据转换为矩阵
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [6]:
rid_to_name, name_to_rid = read_item_names()

toy_story_raw_id = name_to_rid['Now and Then (1995)'] #找到在数据样本中对应的id
toy_story_raw_id

'1053'

In [7]:
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id) #找到在矩阵中的id
toy_story_inner_id

961

In [56]:
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10) #找到最近的几个电影
toy_story_neighbors

[291, 82, 366, 528, 179, 101, 556, 310, 431, 543]

In [9]:
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)


The 10 nearest neighbors of Toy Story are:
While You Were Sleeping (1995)
Batman (1989)
Dave (1993)
Mrs. Doubtfire (1993)
Groundhog Day (1993)
Raiders of the Lost Ark (1981)
Maverick (1994)
French Kiss (1995)
Stand by Me (1986)
Net, The (1995)
